# **Fishing Dataset Folium Map for DL FG OF**

In this notebook we took the max of each fishing method by month and plotted it on a folium map. We had to experiment with alot of different maps and palettes to extract something useful and visiable from the data. Additionally, we faced some challenges in that our region was not the best for some of the fishing types so often the region had very little data in it. This was also due in part by the fact that most of the ocean is not going to have much fishing in it.

**Note:** In order to see the maps you might have to run the code blocks below as it seems the maps dont save overtime on colab.

**Note:** idential methods to this were followed in another notebook for the other fishing methods



# **Necessary Downloads**

In [ ]:
pip install geemap

     |████████████████████████████████| 450kB 5.3MB/s 
     |████████████████████████████████| 143kB 10.8MB/s 
     |████████████████████████████████| 225kB 10.7MB/s 
     |████████████████████████████████| 5.1MB 11.3MB/s 
     |████████████████████████████████| 1.2MB 34.5MB/s 
     |████████████████████████████████| 102kB 8.1MB/s 
     |████████████████████████████████| 1.3MB 37.7MB/s 
     |████████████████████████████████| 81kB 8.0MB/s 
     |████████████████████████████████| 102kB 9.8MB/s 
     |████████████████████████████████| 1.6MB 37.8MB/s 
     |████████████████████████████████| 71kB 7.6MB/s 
     |████████████████████████████████| 122kB 36.7MB/s 
     |████████████████████████████████| 552kB 38.3MB/s 
     |████████████████████████████████| 122kB 40.7MB/s 
     |████████████████████████████████| 378kB 36.8MB/s 
     |████████████████████████████████| 71kB 6.4MB/s 
  Created wheel for pycrs: filename=PyCRS-1.0.2-cp37-none-any.whl size=32683 sha256=ca9bdd0af84bc9674e87c701ea928

In [ ]:
!earthengine authenticate


Instructions for updating:
non-resource variables are not supported in the long term
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=63B8NemNxeloMDt2-rYLxm7FAv15WATuK5zFjdyKgzc&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g6xFa3dGi_BdGXvk97e0DlzYV6mnuni_5Lj0w391fhYnnH9DlILApg

Successfully saved authorization token.


In [ ]:
!pip install earthengine-api
!pip install folium

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import ee
import geemap
ee.Initialize()

import matplotlib.pyplot as plt
import numpy as np

# **Getting the Geometry**

In [ ]:
#from class
import json
import numpy as np
import matplotlib.pyplot as plt
import numpy.ma as ma

jsonFile = '/content/drive/My Drive/ECE471/Final_Project/map.geojson'

with open(jsonFile, 'r') as file:
  jsonLoad = json.load(file)

geometry = jsonLoad['features'][0]['geometry']

In [ ]:
geometry

{'coordinates': [[[-75.146484375, 32.32427558887655],
   [-9.31640625, 32.32427558887655],
   [-9.31640625, 46.49839225859763],
   [-75.146484375, 46.49839225859763],
   [-75.146484375, 32.32427558887655]]],
 'type': 'Polygon'}

# **Folium Map**

Some of the code below is taken from a folium map tutorial. 

In [ ]:
from IPython.display import Image
import folium
from folium import plugins

In [ ]:
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

In [ ]:
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# **Drifting Longlines**

As mentioned each of the methods was seperated by month and then the max composite of each month was plotted. One other thing to note is that only one year was taken as from previous distribution plots made it seems like the data is pretty much consistant throughout the years.

All the other fishing methods followed the same structure for setting up

In [ ]:
datasetDL1 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-01-01', '2016-01-31')).select('drifting_longlines')
datasetDL2 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-02-01', '2016-02-28')).select('drifting_longlines')
datasetDL3 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-03-01', '2016-03-31')).select('drifting_longlines')
datasetDL4 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-04-01', '2016-04-30')).select('drifting_longlines')
datasetDL5 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-05-01', '2016-05-31')).select('drifting_longlines')
datasetDL6 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-06-01', '2016-06-30')).select('drifting_longlines')
datasetDL7 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-07-01', '2016-07-31')).select('drifting_longlines')
datasetDL8 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-08-01', '2016-08-31')).select('drifting_longlines')
datasetDL9 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-09-01', '2016-09-30')).select('drifting_longlines')
datasetDL10 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-10-01', '2016-10-31')).select('drifting_longlines')
datasetDL11 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-11-01', '2016-11-30')).select('drifting_longlines')
datasetDL12 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-12-01', '2016-12-31')).select('drifting_longlines')

In [ ]:
# Set visualization parameters.
# DLVis = {
#   min: 0.0,
#   max: 50.0,
#   "palette": ['white', 'black'],
#   "opacity": 0.85
# }

DLVis = {
  min: 0.0,
  max: 50.0,
  "palette": ['8ab4f8', 'FF0000'],
  "opacity": 0.7
}

points = [(geometry['coordinates'][0][0][1],geometry['coordinates'][0][0][0]),(geometry['coordinates'][0][2][1],geometry['coordinates'][0][2][0])]

#Map.setCenter(20.3, -20.39, 2)
#Map.addLayer(sstMean, visParams, 'Sea Surface Temperature')

# Create a folium map object.
my_map = folium.Map(location=[20.3, -20.39], zoom_start=3, height=500)

# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
basemaps['Google Satellite Hybrid'].add_to(my_map)

# Add layers
Map = geemap.Map(center=[20.3, -20.39], zoom=8)

#Map.setCenter(16.201, 36.316, 7);
my_map.add_ee_layer(datasetDL1.max(), DLVis, 'Drifting Longlines Jan');
my_map.add_ee_layer(datasetDL2.max(), DLVis, 'Drifting Longlines Feb');
my_map.add_ee_layer(datasetDL3.max(), DLVis, 'Drifting Longlines Mar');
my_map.add_ee_layer(datasetDL4.max(), DLVis, 'Drifting Longlines Apr');
my_map.add_ee_layer(datasetDL5.max(), DLVis, 'Drifting Longlines May');
my_map.add_ee_layer(datasetDL6.max(), DLVis, 'Drifting Longlines Jun');
my_map.add_ee_layer(datasetDL7.max(), DLVis, 'Drifting Longlines Jul');
my_map.add_ee_layer(datasetDL8.max(), DLVis, 'Drifting Longlines Aug');
my_map.add_ee_layer(datasetDL9.max(), DLVis, 'Drifting Longlines Sep');
my_map.add_ee_layer(datasetDL10.max(), DLVis, 'Drifting Longlines Oct');
my_map.add_ee_layer(datasetDL11.max(), DLVis, 'Drifting Longlines Nov');
my_map.add_ee_layer(datasetDL12.max(), DLVis, 'Drifting Longlines Dec');

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Display the map.
#my_map.fit_bounds(geometry['coordinates'][0])
folium.Rectangle(bounds=points, color='#ff7800', fill=True, fill_color='#ffff00', fill_opacity=0.01).add_to(my_map)

display(my_map)

in the above map there is a shift throughout the year from southwest to northest and back

# **Fixed Gear**

In [ ]:
datasetFG1 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-01-01', '2016-01-31')).select('fixed_gear')
datasetFG2 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-02-01', '2016-02-28')).select('fixed_gear')
datasetFG3 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-03-01', '2016-03-31')).select('fixed_gear')
datasetFG4 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-04-01', '2016-04-30')).select('fixed_gear')
datasetFG5 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-05-01', '2016-05-31')).select('fixed_gear')
datasetFG6 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-06-01', '2016-06-30')).select('fixed_gear')
datasetFG7 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-07-01', '2016-07-31')).select('fixed_gear')
datasetFG8 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-08-01', '2016-08-31')).select('fixed_gear')
datasetFG9 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-09-01', '2016-09-30')).select('fixed_gear')
datasetFG10 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-10-01', '2016-10-31')).select('fixed_gear')
datasetFG11 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-11-01', '2016-11-30')).select('fixed_gear')
datasetFG12 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-12-01', '2016-12-31')).select('fixed_gear')

In [ ]:
# Set visualization parameters.
DLVis = {
  min: 0.0,
  max: 50.0,
  "palette": ['8ab4f8', 'FF0000'],
  "opacity": 0.7
}

points = [(geometry['coordinates'][0][0][1],geometry['coordinates'][0][0][0]),(geometry['coordinates'][0][2][1],geometry['coordinates'][0][2][0])]

#Map.setCenter(20.3, -20.39, 2)
#Map.addLayer(sstMean, visParams, 'Sea Surface Temperature')

# Create a folium map object.
my_map = folium.Map(location=[20.3, -20.39], zoom_start=3, height=500)

# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
basemaps['Google Satellite Hybrid'].add_to(my_map)

# Add layers
Map = geemap.Map(center=[20.3, -20.39], zoom=8)

#Map.setCenter(16.201, 36.316, 7);
my_map.add_ee_layer(datasetFG1.max(), DLVis, 'Fixed Gear Jan');
my_map.add_ee_layer(datasetFG2.max(), DLVis, 'Fixed Gear Feb');
my_map.add_ee_layer(datasetFG3.max(), DLVis, 'Fixed Gear Mar');
my_map.add_ee_layer(datasetFG4.max(), DLVis, 'Fixed Gear Apr');
my_map.add_ee_layer(datasetFG5.max(), DLVis, 'Fixed Gear May');
my_map.add_ee_layer(datasetFG6.max(), DLVis, 'Fixed Gear Jun');
my_map.add_ee_layer(datasetFG7.max(), DLVis, 'Fixed Gear Jul');
my_map.add_ee_layer(datasetFG8.max(), DLVis, 'Fixed Gear Aug');
my_map.add_ee_layer(datasetFG9.max(), DLVis, 'Fixed Gear Sep');
my_map.add_ee_layer(datasetFG10.max(), DLVis, 'Fixed Gear Oct');
my_map.add_ee_layer(datasetFG11.max(), DLVis, 'Fixed Gear Nov');
my_map.add_ee_layer(datasetFG12.max(), DLVis, 'Fixed Gear Dec');

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Display the map.
#my_map.fit_bounds(geometry['coordinates'][0])
folium.Rectangle(bounds=points, color='#ff7800', fill=True, fill_color='#ffff00', fill_opacity=0.01).add_to(my_map)

display(my_map)

Here we see alot more activity going on near the coast rather than in the middle of the ocean. One thing to note is that during the summer time it seems to be more active near the coast but moves a little bit closer to the coast during the winter times.

# **Other Fishing**

In [ ]:
datasetOF1 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-01-01', '2016-01-31')).select('other_fishing')
datasetOF2 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-02-01', '2016-02-28')).select('other_fishing')
datasetOF3 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-03-01', '2016-03-31')).select('other_fishing')
datasetOF4 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-04-01', '2016-04-30')).select('other_fishing')
datasetOF5 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-05-01', '2016-05-31')).select('other_fishing')
datasetOF6 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-06-01', '2016-06-30')).select('other_fishing')
datasetOF7 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-07-01', '2016-07-31')).select('other_fishing')
datasetOF8 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-08-01', '2016-08-31')).select('other_fishing')
datasetOF9 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-09-01', '2016-09-30')).select('other_fishing')
datasetOF10 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-10-01', '2016-10-31')).select('other_fishing')
datasetOF11 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-11-01', '2016-11-30')).select('other_fishing')
datasetOF12 = ee.ImageCollection('GFW/GFF/V1/fishing_hours').filter(ee.Filter.date('2016-12-01', '2016-12-31')).select('other_fishing')

In [ ]:
# Set visualization parameters.
DLVis = {
  min: 0.0,
  max: 50.0,
  "palette": ['8ab4f8', 'FF0000'],
  "opacity": 0.7
}




points = [(geometry['coordinates'][0][0][1],geometry['coordinates'][0][0][0]),(geometry['coordinates'][0][2][1],geometry['coordinates'][0][2][0])]

#Map.setCenter(20.3, -20.39, 2)
#Map.addLayer(sstMean, visParams, 'Sea Surface Temperature')

# Create a folium map object.
my_map = folium.Map(location=[20.3, -20.39], zoom_start=3, height=500)

# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
basemaps['Google Satellite Hybrid'].add_to(my_map)

# Add layers
Map = geemap.Map(center=[20.3, -20.39], zoom=8)

#Map.setCenter(16.201, 36.316, 7);
my_map.add_ee_layer(datasetOF1.max(), DLVis, 'Other Fishing Jan');
my_map.add_ee_layer(datasetOF2.max(), DLVis, 'Other Fishing Feb');
my_map.add_ee_layer(datasetOF3.max(), DLVis, 'Other Fishing Mar');
my_map.add_ee_layer(datasetOF4.max(), DLVis, 'Other Fishing Apr');
my_map.add_ee_layer(datasetOF5.max(), DLVis, 'Other Fishing May');
my_map.add_ee_layer(datasetOF6.max(), DLVis, 'Other Fishing Jun');
my_map.add_ee_layer(datasetOF7.max(), DLVis, 'Other Fishing Jul');
my_map.add_ee_layer(datasetOF8.max(), DLVis, 'Other Fishing Aug');
my_map.add_ee_layer(datasetOF9.max(), DLVis, 'Other Fishing Sep');
my_map.add_ee_layer(datasetOF10.max(), DLVis, 'Other Fishing Oct');
my_map.add_ee_layer(datasetOF11.max(), DLVis, 'Other Fishing Nov');
my_map.add_ee_layer(datasetOF12.max(), DLVis, 'Other Fishing Dec');

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Display the map.
#my_map.fit_bounds(geometry['coordinates'][0])
folium.Rectangle(bounds=points, color='#ff7800', fill=True, fill_color='#ffff00', fill_opacity=0.01).add_to(my_map)

display(my_map)

Similar pattern as fixed gear fishing is seen here